# Reconocimiento de dígitos con KNN

Veamos como interactuar con las clases implementadas en el módulo de C++ haciendo una experimentación muy simplificada, y a modo de ejemplo, del método KNN para reconocer digitos

## Compilando el módulo desde C++

Definir los path al ejecutable de python 3.6 y sus librerías,
de acuerdo al virtual env que estén corriendo.

In [1]:
!cd .. && mkdir build
!cd ../build/ && rm -rf *
!cd ../build && cmake \
  -DPYTHON_EXECUTABLE="$(which python)" \
  -DCMAKE_BUILD_TYPE=Release ..
!cd ../build && make install

mkdir: cannot create directory ‘build’: File exists
-- The C compiler identification is GNU 9.3.0
-- The CXX compiler identification is GNU 9.3.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
Release mode
-- Found PythonInterp: /home/ariana/mn-tp3/tp3/venv-tp3/bin/python (found version "3.8.10") 
-- Found PythonLibs: /usr/lib/x86_64-linux-gnu/libpython3.8.so
-- pybind11 v2.2.4
-- Performing Test HAS_FLTO
-- Performing Test HAS_FLTO - Success
-- LTO enabled
CMAKE_INSTALL_PREFIX=/home/ariana/mn-tp3/tp3
-- Configuring done
-- Generating done
-

In [1]:
# Verifico la correcta instalación. Si no falla el import está OK
!pwd
!python --version
import metnum

/home/dante/Documents/metnum tps again/mn-tp3/tp3/notebooks
Python 3.8.10


## Explorando los datos

In [2]:
%load_ext autoreload
%autoreload 2
#Tengo problemas con los paths, lo uso por eso pero udstedes no deberian necesitarlo
import sys
sys.path.append('/home/dante/Documents/metnum tps again/mn-tp3/tp3/metnum3/lib/python3.8/site-packages')
import pandas as pd

df_train = pd.read_csv("../dataset-folder/expectativa_de_vida.csv")

Es un csv, con columnas por cada pixel

In [3]:
df_train[:10]

,Unnamed: 0,Country,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,BMI,...,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling,Status
0,0,Afghanistan,58.19375,269.0625,78.2500,0.014375,34.960110,64.562500,2362.2500,15.51875,...,8.252500,52.3125,0.10000,340.015425,9.972260e+06,16.58125,15.58125,0.415375,8.21250,Developing
1,1,Albania,75.15625,45.0625,0.6875,4.848750,193.259091,98.000000,53.3750,49.06875,...,5.945625,98.0625,0.10000,2119.726679,6.969116e+05,1.61875,1.70000,0.709875,12.13750,Developing
2,2,Algeria,73.61875,108.1875,20.3125,0.406667,236.185241,78.000000,1943.8750,48.74375,...,4.604000,91.8750,0.10000,2847.853392,2.164983e+07,6.09375,5.97500,0.694875,12.71250,Developing
3,3,Angola,49.01875,328.5625,83.7500,5.740667,102.100268,70.222222,3561.3125,18.01875,...,3.919333,47.6875,2.36875,1975.143045,1.014710e+07,6.19375,6.66875,0.458375,8.04375,Developing
4,4,Antigua and Barbuda,75.05625,127.5000,0.0000,7.949333,1001.585226,98.266667,0.0000,38.42500,...,4.791333,98.3125,0.12500,9759.305728,NaN,3.42500,3.37500,0.488625,8.84375,Developing
5,5,Argentina,75.15625,106.0000,10.1250,7.966667,773.038981,81.285714,2.0000,54.98125,...,6.912667,92.3750,0.10000,6998.575103,2.012120e+07,1.07500,0.95000,0.794125,16.50625,Developing
6,6,Armenia,73.40000,117.3750,1.0000,3.702667,131.007419,87.562500,274.2500,44.70625,...,4.861333,87.3750,0.10000,1999.986318,1.015165e+06,2.03750,2.11250,0.698375,11.78750,Developing
7,7,Australia,81.81250,63.1875,1.0000,10.155333,5332.226473,93.400000,103.9375,55.86250,...,8.836667,86.8750,0.10000,34637.565047,4.587010e+06,0.66875,0.62500,0.918125,20.03750,Developed
8,8,Austria,81.48125,65.7500,0.0000,12.236000,4928.439188,81.062500,77.2500,48.28750,...,4.715333,86.7500,0.10000,33827.476309,6.474880e+06,1.73125,1.93750,0.862375,15.38750,Developed
9,9,Azerbaijan,70.73125,115.1875,6.0625,0.992000,131.148747,59.214286,598.1875,43.84375,...,5.679333,82.3125,0.10000,3302.947807,2.157370e+06,2.90000,2.94375,0.702687,11.35000,Developing


### Reducir el dataset

Ya que el dataset es grande, me quedo con una porción de éste

In [4]:
df_train = df_train.dropna()
df_train = df_train[:5000]

In [5]:
df_train

,Unnamed: 0,Country,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,BMI,...,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling,Status
0,0,Afghanistan,58.19375,269.0625,78.2500,0.014375,34.960110,64.562500,2362.2500,15.51875,...,8.252500,52.3125,0.10000,340.015425,9.972260e+06,16.58125,15.58125,0.415375,8.21250,Developing
1,1,Albania,75.15625,45.0625,0.6875,4.848750,193.259091,98.000000,53.3750,49.06875,...,5.945625,98.0625,0.10000,2119.726679,6.969116e+05,1.61875,1.70000,0.709875,12.13750,Developing
2,2,Algeria,73.61875,108.1875,20.3125,0.406667,236.185241,78.000000,1943.8750,48.74375,...,4.604000,91.8750,0.10000,2847.853392,2.164983e+07,6.09375,5.97500,0.694875,12.71250,Developing
3,3,Angola,49.01875,328.5625,83.7500,5.740667,102.100268,70.222222,3561.3125,18.01875,...,3.919333,47.6875,2.36875,1975.143045,1.014710e+07,6.19375,6.66875,0.458375,8.04375,Developing
5,5,Argentina,75.15625,106.0000,10.1250,7.966667,773.038981,81.285714,2.0000,54.98125,...,6.912667,92.3750,0.10000,6998.575103,2.012120e+07,1.07500,0.95000,0.794125,16.50625,Developing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,185,Uruguay,76.07500,119.9375,0.5625,6.172667,621.838919,94.312500,0.0000,52.92500,...,8.750000,89.1250,0.10000,7192.584875,2.396771e+06,1.60000,1.54375,0.765625,15.23125,Developing
176,186,Uzbekistan,68.03125,184.8125,21.9375,1.608667,44.373450,95.642857,208.4375,34.80625,...,5.638000,98.4375,0.20625,651.092359,9.036317e+05,3.14375,3.17500,0.603000,11.64375,Developing
177,187,Vanuatu,71.38750,137.8750,0.0000,0.806667,282.325746,56.125000,20.8750,44.25625,...,3.928667,59.0625,0.10000,2000.245518,1.230962e+05,1.56875,1.49375,0.367500,10.56875,Developing
181,191,Zambia,53.90625,354.3125,33.4375,2.239333,89.650407,69.818182,6563.8125,17.45000,...,5.824000,74.2500,11.93125,811.811841,6.260246e+06,6.88125,6.76250,0.498437,11.21250,Developing


In [6]:
# Uso values para mandar todo a arrays de numpy
X = df_train[df_train.columns[1:]].values
y = df_train["Life expectancy"].values.reshape(-1, 1)

X.shape, y.shape

((133, 21), (133, 1))

Pasamos los datos que son de tipo string a float:
- Los países serán enumerados según orden en el data-set.
- Los países con status "developed" serán 1, sino 0.

In [7]:
enum = 0
for row in X:
    row[0] = enum
    row[20] = 1
    if row[20] == "Developing":
        row[20] = 0
    enum+=1

print(X)

[[0 58.19375 269.0625 ... 0.415375 8.212500000000002 1]
 [1 75.15624999999999 45.0625 ... 0.709875 12.137499999999998 1]
 [2 73.61874999999999 108.1875 ... 0.694875 12.712499999999997 1]
 ...
 [130 71.38749999999999 137.875 ... 0.3675 10.568749999999998 1]
 [131 53.90624999999999 354.3125 ... 0.4984375 11.2125 1]
 [132 50.48749999999999 462.375 ... 0.4391249999999999 9.825 1]]


In [ ]:
# Test
#X = [[1,4,7],[2, 84, 8],[3, 6, 9]]
#y = [2,5,12]

In [ ]:
clf = metnum.CML()
x = clf.fit(X, y)
print(x)

In [8]:
clf = metnum.CML()
clf.fit(X[:80], y[:80])
x = clf.predict(X[80:])
print(x)

[[75.71875]
 [65.8875 ]
 [74.5    ]
 [72.15625]
 [53.39375]
 [64.2    ]
 [60.4    ]
 [66.48125]
 [81.13125]
 [73.45   ]
 [56.98125]
 [51.35625]
 [64.5    ]
 [76.4875 ]
 [61.68125]
 [73.1125 ]
 [73.6625 ]
 [67.575  ]
 [75.65   ]
 [79.99375]
 [74.05   ]
 [67.7625 ]
 [59.3125 ]
 [73.61875]
 [65.1625 ]
 [62.56875]
 [73.95625]
 [72.375  ]
 [46.1125 ]
 [67.7125 ]
 [57.5    ]
 [82.06875]
 [73.4    ]
 [70.08125]
 [51.325  ]
 [82.51875]
 [70.85   ]
 [66.65625]
 [73.08125]
 [64.75625]
 [56.6625 ]
 [72.53125]
 [71.06875]
 [74.35625]
 [73.9125 ]
 [64.61875]
 [55.70625]
 [69.9375 ]
 [76.075  ]
 [68.03125]
 [71.3875 ]
 [53.90625]
 [50.4875 ]]


In [9]:
import numpy as np
np.allclose(x,y[80:])

True